<div dir='rtl' style='direction:rtl:'>

# جست و جوی برداری بوسیله ی Elastic Search
در این بخش،
به ساخت دیتابیس،
افزودن امبدینگ های تولید شده
و جست و جوی امبدینگ درخواستی در میان
امبدینگ های ذخیره شده می پردازیم.


In [ ]:
!pip install elasticsearch==7.13.0 transformers tensorflow-io -q
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 913.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow -q
# !pip install tensorflow_text -q
# !pip install tensorflow_hub -q
!pip install elasticsearch_dsl -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 1.2 MB/s eta 0:00:00


In [ ]:
import os
import time
from sklearn.model_selection import train_test_split
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow_io as tfio
import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text as text
from elasticsearch import Elasticsearch, helpers
from elasticsearch_dsl import Search, Q
import hashlib
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm_notebook
import time

In [ ]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.33.0
tensorflow version: 2.12.0


<div dir='rtl' style='direction:rtl:'>

#  تنطیم اولیه Elastic Search



In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
%%bash

ps -ef | grep elasticsearch

root         981     979  0 15:43 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       983     981  7 15:43 ?        00:00:00 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-14600612574893112464 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:fileco

In [ ]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "d24bbb2c6b6c",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "YRullyRiSRKstNGllpqpMw",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
from elasticsearch import Elasticsearch

ES_NODES = "http://localhost:9200/"
es = Elasticsearch(hosts = [ES_NODES])

index_name = 'biobert_embeddings'

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

time.sleep(30)
es.ping()

True

In [ ]:
create_response = es.cat.indices() #print new index list
print(create_response)

yellow open biobert_embeddings peZp2GQtT6G-6vIVYFeWTA 1 1 0 0 208b 208b



In [ ]:
print(es.info())

{'name': '61987699fead', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'vylcWat6Qo-2jqmqYf3Log', 'version': {'number': '7.9.2', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e', 'build_date': '2020-09-23T00:45:33.626720Z', 'build_snapshot': False, 'lucene_version': '8.6.2', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [ ]:
#drive
import gdown
import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1vx89vDIvGtc5w_kGp_T2OtCYDSnvhAEn'})
download.GetContentFile('pubmed_qa_labeled_train.csv')

download = drive.CreateFile({'id': '1fZIhPjl2SOpteFgIZ3l4pMizf-Aht7vL'})
download.GetContentFile('concatenated_embedding_labeled_sentence1.pt')

download = drive.CreateFile({'id': '17Ujs0azWlaaCha6T9F84JQIplOs4xRfr'})
download.GetContentFile('concatenated_articles_labeled_sentence1.csv')

download = drive.CreateFile({'id': '15EJMUm7oUZ-219JlLw9CprxQ0yr0Lk6n'})
download.GetContentFile('sentence2_embedding.pt')

In [ ]:
df_qa = pd.read_csv('/content/pubmed_qa_labeled_train.csv')
concatenated_csv = pd.read_csv("/content/concatenated_articles_labeled_sentence1.csv")

In [ ]:
import numpy as np
import torch
from elasticsearch import Elasticsearch, helpers
import time

es = Elasticsearch()

BODY_INDEX_NAME = "body_vectors"
BATCH_SIZE = 1000
SEARCH_SIZE = 3

df_samples = concatenated_csv
# df_samples.rename(columns={"pubid": "id"}, inplace=True)

embeddings = torch.load("/content/concatenated_embedding_labeled_sentence1.pt")
query_embeddings = torch.load("/content/sentence2_embedding.pt")
paper_ids = df_samples["pubid"].tolist()


def index_data(embeddings, paper_ids):  # Index the embeddings and their associated paper_ids
    body_index_mapping = {
        "mappings": {
            "properties": {
                "body_text_vector": {"type": "dense_vector", "dims": embeddings.shape[1]},
                "paper_id": {"type": "keyword"}
            }
        }
    }

    es.indices.create(index=BODY_INDEX_NAME, body=body_index_mapping, ignore=400)

    requests = []
    for embedding, paper_id in zip(embeddings, paper_ids):
        request = {
            "_op_type": "index",
            "_index": BODY_INDEX_NAME,
            "body_text_vector": embedding.tolist(),
            "paper_id": paper_id
        }
        requests.append(request)

    indexing_start = time.time()
    helpers.bulk(es, requests)
    es.indices.refresh(index=BODY_INDEX_NAME)
    indexing_time = time.time() - indexing_start

    print("Done indexing. Indexing time: {:.2f} seconds.".format(indexing_time))

def get_random_query_embedding():
    query_embeddings = torch.load("/content/sentence2_embedding.pt")
    question_number = np.random.randint(len(query_embeddings))
    return np.array(query_embeddings[question_number]), question_number

def run_query_loop():
    for i in range(1):
        try:
            handle_query()
        except KeyboardInterrupt:
            return

def handle_query():
    # take embedding
    query_embeddings = torch.load("/content/sentence2_embedding.pt")
    question_number = np.random.randint(len(query_embeddings))
    query_embedding = np.array(query_embeddings[question_number])

    search_start = time.time()

    response = es.search(
        index=BODY_INDEX_NAME,
        body={
            "size": BATCH_SIZE,
            "query": {"match_all": {}},
            "_source": {"includes": []},
            "docvalue_fields": ["paper_id", "body_text_vector"]
        }
    )

    search_time = time.time() - search_start

    results = []
    for hit in response["hits"]["hits"][:SEARCH_SIZE]:
        doc_vector = np.array(hit["fields"]["body_text_vector"])

        query_embedding = query_embedding.reshape(1, -1)
        doc_vector = doc_vector.reshape(1, -1)



        similarity = np.dot(query_embedding, doc_vector.T) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_vector))

        print(query_embedding)
        print(doc_vector)
        print(similarity)


        results.append((hit["fields"]["paper_id"][0], similarity[0][0]))

    results.sort(key=lambda x: x[1], reverse=True)

    print()
    print("{} total hits.".format(len(results)))
    print("search time: {:.2f} ms".format(search_time * 1000))

    question_text = df_qa.loc[question_number, "question"]
    print("Query: ", question_text)
    print("\n")

    for paper_id, similarity in results[:SEARCH_SIZE]:
        abstract = df_samples[df_samples["pubid"] == paper_id]["context"].values[0]
        print("abstract:", abstract)
        print("paper_id: {}, similarity: {:.4f}".format(paper_id, similarity))
        print("\n\n")


In [ ]:

index_data(embeddings, paper_ids)

In [ ]:
run_query_loop()